# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_")  and "puebla" in nombre and '2024' in nombre
    and 'q' not in nombre
    and 'financiamientos' not in nombre
    and 'poblacion' not in nombre
    and 'salarios' not in nombre
    and 'publicacion' not in nombre
    
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla',
 'df_mar_2024_puebla',
 'df_may_2024_puebla']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','Metros_total':'metros_total','m_total':'metros_total','m_construido':'metros_construido','Metros_construido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total','Precio_M2_Construido':'precio_m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [5]:
df_total.columns

Index(['categoria', 'propiedad', 'precio', 'metros_total', 'metros_construido',
       'precio_m2_construido', 'estacionamiento', 'recamaras', 'baño',
       'medio_baño', 'baño_total'],
      dtype='object')

In [6]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   categoria             27934 non-null  object 
 1   propiedad             27934 non-null  object 
 2   precio                27934 non-null  float64
 3   metros_total          27934 non-null  int64  
 4   metros_construido     27934 non-null  int64  
 5   precio_m2_construido  27934 non-null  float64
 6   estacionamiento       27934 non-null  int64  
 7   recamaras             27934 non-null  int64  
 8   baño                  27934 non-null  int64  
 9   medio_baño            27934 non-null  int64  
 10  baño_total            27934 non-null  float64
dtypes: float64(3), int64(6), object(2)
memory usage: 2.3+ MB


In [7]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
#def asignar_categoria(precio):
#    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
#        if limite_inferior <= precio < limite_superior:
#            return categoria
#    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return "Sin categoría"  # Asignar una etiqueta clara en lugar de None
df_total = df_total.drop(columns=['categoria'], errors='ignore')


# Convertir 'precio' a numérico y manejar errores para tener la categoría correcta 
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
# Llenar valores faltantes con un número por defecto, por ejemplo, 0
df_total['precio'] = df_total['precio'].fillna(0).astype(int)
df_total = df_total[df_total['precio'] > 0]


df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])


In [8]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=10) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,)#Mínimo 1 #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

df_total = df_total[df_total['precio'] != 0]
#df_total = df_total[df_total['precio'] > 4800]
df_total = df_total[df_total['metros_total'] > 20]
df_total = df_total[df_total['metros_construido'] > 20]

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

# Casa promedio

In [9]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_promedios = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)



print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)


# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedioo.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      4.0        4.0    3.0
S                      3.0        3.0    3.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      4.0        4.0    3.0
S                      3.0        3.0    3.0
general                2.0        3.0    2.0


# Casa Mediana

In [10]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_mediana = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
grupo_mediana.astype(int)
# Renombrar las columnas para mayor claridad
grupo_mediana.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
print(grupo_mediana)

# Calcular mediana de estacionamientos, recámaras y baños en general (sin segmento específico)
mediana_general = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
mediana_general = mediana_general.astype(int)
# Renombrar las columnas para mayor claridad
mediana_general.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
mediana_general = pd.DataFrame(mediana_general).T
mediana_general['segmento'] = 'general'
print(mediana_general)

casa_media = pd.concat([grupo_mediana, mediana_general.set_index('segmento')], ignore_index=False)
print(casa_media)

# Guardar el JSON en un archivo
grupo_promedios_json = casa_media.to_json(orient='index')
with open('assets/tablas/grupo_casa_mediana.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      5.0        4.0    3.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      5.0        4.0    3.0
S                      2.0        3.0    2.0
general                2.0        3.0    2.0


# Precio Mercado

In [11]:
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean','50%']] #Columnas deseadas
grupo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,3252000.0,3999000.0,3.645082e+06,3667939.0
1,B,2502235.0,3249000.0,2.879851e+06,2850000.0
2,C,1754500.0,2499000.0,2.135168e+06,2170000.0
3,D,1005000.0,1749000.0,1.380241e+06,1375000.0
4,E,148640.0,335410000.0,2.463555e+06,700000.0
5,L,12250000.0,21995000.0,1.546130e+07,15000000.0
6,S,750000.0,22000000.0,5.722922e+06,5035499.0


In [12]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,252,000.00
1     $2,502,235.00
2     $1,754,500.00
3     $1,005,000.00
4       $148,640.00
5    $12,250,000.00
6       $750,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $3,999,000.00
1      $3,249,000.00
2      $2,499,000.00
3      $1,749,000.00
4    $335,410,000.00
5     $21,995,000.00
6     $22,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,252,000.00","$3,999,000.00","$3,645,082.27","$3,667,939.00"
1,B,"$2,502,235.00","$3,249,000.00","$2,879,851.13","$2,850,000.00"
2,C,"$1,754,500.00","$2,499,000.00","$2,135,167.59","$2,170,000.00"
3,D,"$1,005,000.00","$1,749,000.00","$1,380,240.97","$1,375,000.00"
4,E,"$148,640.00","$335,410,000.00","$2,463,555.15","$700,000.00"
5,L,"$12,250,000.00","$21,995,000.00","$15,461,298.26","$15,000,000.00"
6,S,"$750,000.00","$22,000,000.00","$5,722,921.80","$5,035,499.00"


In [13]:
todo_descripcion = df_total['precio'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $148,640.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $335,410,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will ra

,Mínimo,Máximo,Promedio,Mediana,segmento
precio,"$148,640.00","$335,410,000.00","$3,831,041.97","$2,490,000.00",general


In [14]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [15]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,252,000.00","$3,999,000.00","$3,645,082.27","$3,667,939.00"
1,B,"$2,502,235.00","$3,249,000.00","$2,879,851.13","$2,850,000.00"
2,C,"$1,754,500.00","$2,499,000.00","$2,135,167.59","$2,170,000.00"
3,D,"$1,005,000.00","$1,749,000.00","$1,380,240.97","$1,375,000.00"
4,E,"$148,640.00","$335,410,000.00","$2,463,555.15","$700,000.00"
5,L,"$12,250,000.00","$21,995,000.00","$15,461,298.26","$15,000,000.00"
6,S,"$750,000.00","$22,000,000.00","$5,722,921.80","$5,035,499.00"
7,general,"$148,640.00","$335,410,000.00","$3,831,041.97","$2,490,000.00"


In [16]:
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]

# Función para limpiar valores con unidades y convertirlos a float
def limpiar_valor(valor):
    # Eliminar unidades (m², $, etc.) y convertir a float
    if isinstance(valor, str):
        return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))
    else:
        return valor

# Aplicar limpieza de valores en las columnas que contienen medidas
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)
# Función para formatear nuevamente los valores en el formato deseado
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Volver a aplicar el formato monetario después de los ajustes
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)



In [17]:
# Limpiar valores y convertir a float
def limpiar_valor(valor):
    return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)

# Verificar el orden de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]
resultados_finales_descripcion = resultados_finales_descripcion.set_index('segmento').reindex(orden_segmentos).reset_index()

# Validar que los valores no sean inconsistentes
for i in range(1, len(resultados_finales_descripcion)):
    for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
        valor_actual = resultados_finales_descripcion.loc[i, columna]
        valor_anterior = resultados_finales_descripcion.loc[i - 1, columna]
        
        # Ajustar valores inconsistentes
        if valor_actual < valor_anterior:
            resultados_finales_descripcion.loc[i, columna] = valor_anterior

# Volver a formatear los valores a formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)

# Guardar los resultados en un archivo JSON
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='records', force_ascii=False)
with open('assets/tablas/grupo_precio_mercado_ajustado.json', 'w', encoding='utf-8') as f:
    f.write(resultados_finales_desc_json)


In [18]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,E,"$148,640.00","$335,410,000.00","$2,463,555.15","$700,000.00"
1,D,"$1,005,000.00","$335,410,000.00","$2,463,555.15","$1,375,000.00"
2,C,"$1,754,500.00","$335,410,000.00","$2,463,555.15","$2,170,000.00"
3,B,"$2,502,235.00","$335,410,000.00","$2,879,851.13","$2,850,000.00"
4,A,"$3,252,000.00","$335,410,000.00","$3,645,082.27","$3,667,939.00"
5,S,"$3,252,000.00","$335,410,000.00","$5,722,921.80","$5,035,499.00"
6,L,"$12,250,000.00","$335,410,000.00","$15,461,298.26","$15,000,000.00"
7,ELITE,$nan,$nan,$nan,$nan
8,general,"$148,640.00","$335,410,000.00","$3,831,041.97","$2,490,000.00"


In [19]:

# Guardar el JSON con los resultados finales
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='records', force_ascii=False)
with open('assets/tablas/grupo_precio_mercado_ordenado.json', 'w', encoding='utf-8') as f:
    f.write(resultados_finales_desc_json)


# M2 Terreno

In [20]:
df_total_terreno = df_total[df_total['metros_total'] != 0]
df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1) & (df_total_terreno['metros_total'] > 20)]

df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\1389371296.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    50.00 m²
1    43.00 m²
2    28.00 m²
3    33.00 m²
4    30.00 m²
5    80.00 m²
6    34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\1389371296.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       22,715.00 m²
1       26,964.00 m²
2       19,124.00 m²
3       12,289.00 m²
4      220,000.00 m²
5        9,288.00 m²
6    7,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i]

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,50.00 m²,"22,715.00 m²",273.29 m²,140.00 m²
1,B,43.00 m²,"26,964.00 m²",272.16 m²,124.00 m²
2,C,28.00 m²,"19,124.00 m²",276.67 m²,102.00 m²
3,D,33.00 m²,"12,289.00 m²",156.45 m²,97.00 m²
4,E,30.00 m²,"220,000.00 m²",333.05 m²,150.00 m²
5,L,80.00 m²,"9,288.00 m²",837.64 m²,682.00 m²
6,S,34.00 m²,"7,500,000.00 m²","2,442.58 m²",220.00 m²


In [21]:
df_total_terreno = df_total[df_total['metros_total'] != 0]
df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1)&(df_total_terreno['metros_total'] > 20)]

todo_descripcion = df_total_terreno[df_total_terreno['metros_total'] != 0]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\2712485081.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    28.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\2712485081.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    7,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\2712485081.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,28.00 m²,"7,500,000.00 m²",904.43 m²,149.00 m²,general


In [22]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [23]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,50.00 m²,"22,715.00 m²",273.29 m²,140.00 m²
1,B,43.00 m²,"26,964.00 m²",272.16 m²,124.00 m²
2,C,28.00 m²,"19,124.00 m²",276.67 m²,102.00 m²
3,D,33.00 m²,"12,289.00 m²",156.45 m²,97.00 m²
4,E,30.00 m²,"220,000.00 m²",333.05 m²,150.00 m²
5,L,80.00 m²,"9,288.00 m²",837.64 m²,682.00 m²
6,S,34.00 m²,"7,500,000.00 m²","2,442.58 m²",220.00 m²
7,general,28.00 m²,"7,500,000.00 m²",904.43 m²,149.00 m²


# M2 Construcción

In [24]:
df_total_terreno = df_total[(df_total['metros_construido'] > 40) & (~df_total['metros_construido'].isin([0, 1,2,3]))]
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\2517475487.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     60.00 m²
1     58.00 m²
2     28.00 m²
3     33.00 m²
4     30.00 m²
5    120.00 m²
6     34.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\2517475487.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       17,468.00 m²
1       14,583.00 m²
2       40,642.00 m²
3       12,289.00 m²
4      220,000.00 m²
5        9,288.00 m²
6    7,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.ilo

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,60.00 m²,"17,468.00 m²",287.29 m²,200.00 m²
1,B,58.00 m²,"14,583.00 m²",257.83 m²,146.00 m²
2,C,28.00 m²,"40,642.00 m²",303.60 m²,119.00 m²
3,D,33.00 m²,"12,289.00 m²",160.57 m²,101.50 m²
4,E,30.00 m²,"220,000.00 m²",321.13 m²,180.00 m²
5,L,120.00 m²,"9,288.00 m²",649.59 m²,566.00 m²
6,S,34.00 m²,"7,500,000.00 m²","2,409.87 m²",270.00 m²


In [25]:
todo_descripcion = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    28.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    7,500,000.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\641405135.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will ra

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,28.00 m²,"7,500,000.00 m²",904.43 m²,149.00 m²,general


In [26]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [27]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,60.00 m²,"17,468.00 m²",287.29 m²,200.00 m²
1,B,58.00 m²,"14,583.00 m²",257.83 m²,146.00 m²
2,C,28.00 m²,"40,642.00 m²",303.60 m²,119.00 m²
3,D,33.00 m²,"12,289.00 m²",160.57 m²,101.50 m²
4,E,30.00 m²,"220,000.00 m²",321.13 m²,180.00 m²
5,L,120.00 m²,"9,288.00 m²",649.59 m²,566.00 m²
6,S,34.00 m²,"7,500,000.00 m²","2,409.87 m²",270.00 m²
7,general,28.00 m²,"7,500,000.00 m²",904.43 m²,149.00 m²


# Pecio M2 Construcción

In [28]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3])]
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]

grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe().reset_index()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_filtrado[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\3813107853.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      $213.98
1      $205.00
2       $44.29
3      $137.52
4      $128.06
5    $1,399.66
6       $87.83
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\3813107853.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $57,716.67
1     $48,710.08
2     $86,250.00
3     $45,945.95
4    $793,310.42
5    $165,833.33
6    $135,714.29
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).app

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,$213.98,"$57,716.67","$21,676.53","$18,624.10"
1,B,$205.00,"$48,710.08","$19,938.89","$19,271.89"
2,C,$44.29,"$86,250.00","$18,057.26","$17,594.34"
3,D,$137.52,"$45,945.95","$13,273.38","$13,366.34"
4,E,$128.06,"$793,310.42","$7,898.52","$4,146.09"
5,L,"$1,399.66","$165,833.33","$28,511.68","$26,000.00"
6,S,$87.83,"$135,714.29","$21,200.27","$19,060.61"


In [29]:
df_total_terreno = df_total_terreno[df_total_terreno['precio_m2_construido'] > 10]
todo_descripcion = df_total_terreno[~df_total_terreno['metros_construido'].isin([0, 1,2,3])]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\3521683632.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $44.29
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\3521683632.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $793,310.42
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_18180\3521683632.py:14: FutureWarning: Setting an item of incompatible dtype is dep

,Mínimo,Máximo,Promedio,Mediana,segmento
precio_m2_construido,$44.29,"$793,310.42","$16,846.93","$16,164.68",general


In [30]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [31]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,$213.98,"$57,716.67","$21,676.53","$18,624.10"
1,B,$205.00,"$48,710.08","$19,938.89","$19,271.89"
2,C,$44.29,"$86,250.00","$18,057.26","$17,594.34"
3,D,$137.52,"$45,945.95","$13,273.38","$13,366.34"
4,E,$128.06,"$793,310.42","$7,898.52","$4,146.09"
5,L,"$1,399.66","$165,833.33","$28,511.68","$26,000.00"
6,S,$87.83,"$135,714.29","$21,200.27","$19,060.61"
7,general,$44.29,"$793,310.42","$16,846.93","$16,164.68"
